In [1]:
from my_functions.assel_functions import *
from my_functions.angel_functions import *

## Optimization Task

Optimization set up

In [ ]:
input_params_names = ['beam_q','beam_z_head','beam_length','n_p_start','n_p_end','l_plateau','E_laser','tau_laser','head_current']

In [ ]:
task = "Task3"
optim_task = 'sc18'
path2sim = f'/beegfs/desy/group/mpa/mpa1/ayermek/{task}/{optim_task}/ensemble'

First, we look at the convergence of energy efficiency during optimization

In [ ]:
N_sim = 1500
best_sim,obj_func = look_objective('Energy_conversion',path2sim,N_sim,plot=True)

Then, we can look at the convergence of the objective function during optimization

In [ ]:
N_sim = 2000
best_sim,obj_func = look_objective('f',path2sim,N_sim,plot=True)

Below calculation of energy conversion of the best simulation based on objective function

In [ ]:
onlyfiles = [f for f in listdir(path2sim)]

folder = f'sim{best_sim}_worker*'
folder = find_folder(folder,onlyfiles)
print(folder)

obj_name = 'Energy_conversion'

Energy_conversion = open(f"{path2sim}/{folder}/{obj_name}.txt", "r")
Energy_conversion = Energy_conversion.read().split('\n')
Energy_conversion = float(Energy_conversion[0])
print(f"Energy_conversion of the best sim = {-1*Energy_conversion} %")

Here we analyze how each parameter converged individually.

In the plots below:
- xaxis: value of the objective function of N best simulations in descending order. So it's best N simulation from left to right
- yaxis: parameter values with respect to best N simulations

In [ ]:
first_N_simulations = 1000
param_analysis(obj_func,first_N_simulations,input_params_names,path2sim)

Analysis of the best simulation shows:

- value of all parameters
- Mean energy of the beam at each iteration
- Plots: 
    - phase-space at each iteration
    - longitudinal profile of the electric field
    - plasma electron density and beam profile at each iteration
    - a0 plot

In [ ]:
best_sim_analysis(path2sim,best_sim,input_params_names,from_iter=0,till_iter=9)

### Resolution scan

In [ ]:
input_params_names = ['beam_q','beam_z_head','beam_length','n_p_start','n_p_end','l_plateau','E_laser','tau_laser','head_current']
variables = ['dr','dz','dz_fields']

opt_task = 'sc18'
res_scan_task = 'rs1'
path2task = f'/beegfs/desy/group/mpa/mpa1/ayermek/Task3/{opt_task}/resolution_scan'

#### dz_fields

Laser energy convergence analysis

In [ ]:
variable = variables[2] # dz_fields
# variable = variables[1] # dz
# variable = variables[0] # dr
valuerange = [1,4,8,16]
folder = f'{variable}_change'
for value in valuerange:
    path2diags = f'{path2task}/{folder}/{variable}_{value}'
    E_laser = analyze_sim(path2dir=f'{path2diags}',plot=False)
    plt.plot(E_laser,label=f"{variable}_{value}")      

plt.xlabel('Iteration')
plt.ylabel('Energy [J]')
plt.legend()         

Time consumption analysis

In [ ]:
folder = f'{variable}_change'
time_all = []
for value in valuerange:
    path2diags = f'{path2task}/{folder}/{variable}_{value}'
    time_ = open(f"{path2diags}/time_.txt", "r")
    time_ = time_.read().split('\n')
    time_ = float(time_[0])
    time_ = time_ / 60
    time_all.append(time_)
    plt.scatter(time_,value,label =f"{variable}_{value}" )
    
plt.plot(time_all,valuerange)
plt.ylabel('dz')
plt.xlabel('Time [minutes]')
plt.legend()
plt.grid()


Best simulation results with different resolutions

In [ ]:
variable_range = valuerange
N_plots = 3 # after Ez plot

from_iter = 4
rows = 1 + len(variable_range) * N_plots
columns = 10 - from_iter
fig = plt.figure(figsize=(5*columns, 3*rows))
grid = plt.GridSpec(rows, columns, wspace = .25, hspace = .25)
N_iterations = 10 - from_iter


for index,iteration in enumerate(range(from_iter,10)):
    exec (f"plt.subplot(grid{[columns * 0 + index]})")
    folder = f'{variable}_change'
    for value in variable_range:
        path2diags = f'{path2task}/{folder}/{variable}_{value}'   

        ts = OpenPMDTimeSeries(f'{path2diags}/diags/hdf5/', backend='h5py')
        Ez, m = ts.get_field(iteration=iteration, field='E', coord='z')
        plt.plot(m.z, Ez[Ez.shape[0]//2,:],label=f"{variable}_{value}")

    plt.ylabel('Ez [MeV]')
    plt.xlabel('z [m]')
    plt.legend() 
    plt.title(f"iter = {iteration}")
    
    print(f'iteration = {iteration}')
    print("--"*20)
    
    for i,value in enumerate(variable_range):
        path2diags = f'{path2task}/{folder}/{variable}_{value}' 
        ts = OpenPMDTimeSeries(f'{path2diags}/diags/hdf5/', backend='h5py')
        
        x, z, ux, uz = ts.get_particle(iteration=iteration, var_list=['x', 'z', 'ux', 'uz'], species='bunch')
        print(f"{variable}_{value}: Energy [MeV]", .511*(np.mean(uz)-1))
        F, m = ts.get_field(iteration=iteration, field='E', coord='z')
        L, m = ts.get_field(iteration=iteration, field='a_mod')
        
        exec (f"plt.subplot(grid{[columns * (len(variable_range)*0 + (i+1)) + index]})")
        plt.imshow(F, extent=m.imshow_extent, aspect='auto')
        plt.clim(-1.e11,1.e11)
        plt.colorbar()
        
        plt.plot(z,x,'k.',ms=.1)
        plt.xlim([m.zmin, m.zmax])
        plt.ylim([m.rmin, m.rmax])
        plt.grid()
        plt.title(f"{variable} = {value}")
        
        exec (f"plt.subplot(grid{[columns * (len(variable_range)*1 + (i+1)) + index]})")
        plt.imshow(L, extent=m.imshow_extent, aspect='auto')
        plt.colorbar()
        
        plt.plot(z,x,'k.',ms=.1)
        plt.xlim([m.zmin, m.zmax])
        plt.ylim([m.rmin, m.rmax])
        plt.grid()
        plt.title(f"{variable} = {value}")
        
        exec (f"plt.subplot(grid{[columns * (len(variable_range)*2 + (i+1)) + index]})")
        above150_uz = []
        above150_z = []
        below150_uz = []
        below150_z = []
        for i in range(len(uz)):
            if (uz[i]-1)*0.511 >= 150:
                above150_uz.append(uz[i])
                above150_z.append(z[i])
            else:
                below150_uz.append(uz[i])
                below150_z.append(z[i]) 
                
        Ez, m = ts.get_field(iteration=iteration, field='E', coord='z')
        
        plt.plot(above150_z,above150_uz,'.',ms=0.5,color='orange', label = 'above 150 MeV')
        plt.plot(below150_z,below150_uz,'.',ms=0.5,color='#beab9e',label = 'below 150 MeV')
        plt.grid()
        plt.xlabel('z [m]')
        plt.ylabel('uz [$m_e \cdot c$]')
        plt.title(f"{variable} = {value}")
        plt.xlim([m.zmin, m.zmax])

        
    print("--"*20)
        
        